In [1]:
# Import packages
import pandas as pd
import numpy as np
import xml.etree.cElementTree as et
import spacy
import re
from nltk import tokenize
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

C:\Users\ConquerV\AppData\Local\Temp/ipykernel_9576/1898551232.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


### 0.Preparing Datalist in Working directory(Jupyter)

In [ ]:
import os
os.chdir('/home/ec2-user/SageMaker/data/Google_Dataset')
cwd = os.getcwd()
# store paths to all data xml files in a list for iteration
all_path = [os.path.join(cwd, f) for f in os.listdir(cwd)]

## 1. Extract Metadata: title, publish date, etc.

In [13]:
def extract_meta(path):
    tree=et.parse(path)
    root=tree.getroot()

    meta = {'Keyword': [], 
            'GOID': [], 
            'Title': [], 
            'Contributors': [],
            'Contributor FirstName': [],
            'Contributor LastName': [],
            'Numeric Date': [],
            'Start Date': [],
            'End Date': [],
            'Qualifier': [],
            'Publisher Country': [],
            'Language': [],
            'Source': [],
            'StartPage': [],
            'DocSection': [],
            'ColumnHeader': [],
            'DocEdition': [],
            'Title Keywords': [],
            'Terms': [],
            'Word Count': [],
            'Title of Newspaper': [],
            'Subjects': [],
            'MpubId': []
            }

    # iteratre over the trees to extract metadata
    for item in root.iter('GOID'):
        meta['GOID'].append(item.text)

    for item in root.iter('TitleAtt'):
        meta['Title'].append(item[0].text)

    for item in root.iter('PubFrosting'):
        for title in item.iter('Qualifier'):
            meta['Qualifier'].append(title.text)
            break

    for item in root.iter('ISO'):
        meta['Language'].append(item[1].text.strip())

    # Contributor Info
    for item in root.iter('Contributors'):
        for contributor in item:
            if not meta['Contributors']:
                meta['Contributors'].append(contributor[0][0].text)
            else:
                meta['Contributors'].append('^^'+contributor[0][0].text)

    for item in root.iter('LastName'):
        if not meta['Contributor LastName']:
            meta['Contributor LastName'].append(item.text)
        else:
            meta['Contributor LastName'].append('^^'+item.text)

    for item in root.iter('FirstName'):
        if not meta['Contributor FirstName']:
            meta['Contributor FirstName'].append(item.text)
        else:
            meta['Contributor FirstName'].append('^^'+item.text)

    for item in root.iter('NumericDate'):
        meta['Numeric Date'].append(item.text)

    for item in root.iter('SourceType'):
        meta['Source'].append(item.text)

    for item in root.iter('Country'):
        meta['Publisher Country'].append(item.text)
    
    # PrintLocation Info
    for item in root.iter('StartPage'):
        meta['StartPage'].append(item.text)

    for item in root.iter('DocSection'):
        meta['DocSection'].append(item.text)
    
    for item in root.iter('ColumnHeader'):
        meta['ColumnHeader'].append(item.text)
    
    for item in root.iter('DocEdition'):
        meta['DocEdition'].append(item.text)
    
    for title in root.iter('Terms'):
        terms = [title.tag, title.attrib, title.text.strip()]
        for item in title:
            item_info = [item.tag, item.attrib, item.text.strip()]
            for sub in item:
                item_info.append(sub.tag)
                item_info.append(sub.attrib)
                item_info.append(sub.text)
            terms.append(item_info)
        meta['Terms'].append(terms)

    # for title in root.iter('GenSubjTerm'):
    #     terms = [title.tag, title.attrib, title.text.strip()]
    #     for item in title:
    #         terms.append(item.tag)
    #         terms.append(item.attrib)
    #     meta['Subject Terms'].append(item.text)

    for item in root.iter('PubFrosting'):
        for title in item.iter('Title'):
            meta['Title of Newspaper'].append(item[2].text)
            break
    
    for item in root.iter('PubFrosting'):
        for title in root.iter('Subjects'):
            meta['Subjects'].append(title[0].text)

    for item in root.iter('TextInfo'):
        for title in item:
            meta['Word Count'].append(title.attrib['WordCount'])
    # df_meta = pd.DataFrame(data=meta)

    for item in root.iter('PubFrosting'):
        for title in root.iter('MpubId'):
            meta['MpubId'].append(title.text)

    for item in root.iter('PubFrosting'):
        for title in root.iter('StartDate'):
            meta['Start Date'].append(title.text)
            break

    for item in root.iter('PubFrosting'):
        for title in root.iter('EndDate'):
            meta['End Date'].append(title.text)
            break
            
    # add na if any header info is not found
    for k in meta:
        if not meta[k]:
            meta[k].append('n/a')

    return meta

path = 'RE_Sample_XMLs/2123327034.xml'
meta = extract_meta(path)

meta_df = pd.DataFrame(data=meta)
meta_df.head()

,Keyword,GOID,Title,Contributors,Contributor FirstName,Contributor LastName,Numeric Date,Start Date,End Date,Qualifier,...,StartPage,DocSection,ColumnHeader,DocEdition,Title Keywords,Terms,Word Count,Title of Newspaper,Subjects,MpubId
0,n/a,2123327034,Always Tackling 'The Hard Problem',"Soloski, Alexis",Alexis,Soloski,2018-10-21,2018-10-21,2018-10-21,"New York, N.Y.",...,AR.4,AR,n/a,n/a,n/a,"[Terms, {}, , [CompanyTerm, {'TermSource': 'AutoCatNstein'}, , CompanyNameAtt, {'TermVocab': 'CORP'}, \n , CompanyNAICAtt, {}, \n , CompanyNAICAtt, {}, \n , CompanyNAICAtt, {}, \n , CompanyNAICAtt, {}, \n ], [CompanyTerm, {'TermSource': 'AutoCatNstein'}, , CompanyNameAtt, {'TermVocab': 'CORP'}, \n , CompanyNAICAtt, {}, \n ], [GenSubjTerm, {'TermSource': 'AutoCatNstein', 'TermVocab': 'PQSUBJ'}, , GenSubjValue, {}, Neurosciences], [GenSubjTerm, {'TermSource': 'AutoCatNstein', 'TermVocab': 'PQSUBJ'}, , GenSubjValue, {}, Consciousness], [GenSubjTerm, {'TermSource': 'AutoCatNstein', 'TermVocab': 'PQSUBJ'}, , GenSubjValue, {}, Theater], [GenSubjTerm, {'TermSource': 'AutoCatNstein', 'TermVocab': 'PQSUBJ'}, , GenSubjValue, {}, Philosophy], [GenSubjTerm, {'TermSource': 'AutoCatNstein', 'TermVocab': 'PQSUBJ'}, , GenSubjValue, {}, Philosophers], [Term, {'TermSource': 'AutoCatNstein', 'TermVocab': 'PERS'}, , Personal, {}, Stoppard, Tom], [Term, {'TermSource': 'AutoCatNstein', 'TermVocab': 'GEOG'}, , Geographic, {}, New York]]",1639,New York Times,General Interest Periodicals--United States,11561


In [48]:
for title in root.iter('CompanyTerm'):
    print(title.tag, title.attrib, title.text)
    for item in title:
        print(item.tag, item.attrib)

CompanyTerm {'TermSource': 'NA', 'PndxHeader': 'true'}          
            
CompanyNameAtt {'TermVocab': 'CORP'}
CompanyNAICAtt {}


In [7]:
meta

{'Keyword': [],
 'GOID': ['2123327034'],
 'Title': ["Always Tackling 'The Hard Problem'"],
 'Contributors': ['Soloski, Alexis'],
 'Contributor FirstName': ['Alexis'],
 'Contributor LastName': ['Soloski'],
 'Numeric Date': ['2018-10-21'],
 'Start Date': ['2018-10-21', '18570101', '2018-10-21'],
 'End Date': ['2018-10-21', '99991231', '2018-10-21'],
 'Qualifier': ['New York, N.Y.', 'New York, N.Y.'],
 'Publisher Country': ['United States'],
 'Language': ['English'],
 'Source': ['Newspapers'],
 'StartPage': ['AR.4'],
 'DocSection': [],
 'ColumnHeader': [],
 'DocEdition': [],
 'Title Keywords': [],
 'Terms': [['Terms',
   {},
   '',
   ['CompanyTerm',
    {'TermSource': 'AutoCatNstein'},
    '',
    'CompanyNameAtt',
    {'TermVocab': 'CORP'},
    '\n               ',
    'CompanyNAICAtt',
    {},
    '\n               ',
    'CompanyNAICAtt',
    {},
    '\n               ',
    'CompanyNAICAtt',
    {},
    '\n               ',
    'CompanyNAICAtt',
    {},
    '\n               '],
   [

## 2. Textfield Parsing & Analysis

### 2.1 Text metadata

In [28]:
# Parse the xml by paragraph and sentence
def extract_paragraphs_new(path):
    tree=et.parse(path)
    root=tree.getroot()
    del_list = []
    full = ''
    for text in root.iter('Text'):
        full = text.text
    for i in range(len(full)):
        if full[i:i+3] == '<p>':
            del_list.append(i)
    temp = {'Paragraph': [], 'Sentence': [], 'Overall Sentence': [], 'Text': []}
    sentence_count = 1
    #temp = {'Paragraph': [], 'Text': []}
    for i in range(len(del_list)-1):
        text = full[(del_list[i]+3):(del_list[i+1])]
        text = text.replace("\n","")
        text = text.replace("<p>","")
        text = text.replace("</p>","")
        text = text.replace("<i>","")
        text = text.replace("</i>","")
        text = text.replace("<b>","")
        text = text.replace("</b>","")
        all_sen = tokenize.sent_tokenize(text)
        for j in range(len(all_sen)):
            temp['Paragraph'].append(i+1)
            temp['Sentence'].append(j+1)
            temp['Overall Sentence'].append(sentence_count)
            temp['Text'].append(all_sen[j])
            sentence_count += 1
    df = pd.DataFrame(data=temp)
    return df

### 2.2 Sentiment Analysis (spaCY)

In [15]:
# for sentiment analysis
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

In [16]:
def get_tags(sentence):
    """Return POS tags of each sentence."""
    res = []
    for token in nlp(sentence):
        res.append((token.text, token.pos_, token.dep_))
    return res

In [17]:
def get_polarity(sentence):
    """Return polarity score of each sentence."""
    token = nlp(sentence)
    polarity = token._.blob.polarity
    return polarity

In [18]:
def get_subjectivity(sentence):
    """Return subjectivity score of each sentence."""
    token = nlp(sentence)
    subjectivity = token._.blob.subjectivity
    return subjectivity

In [19]:
def extract_tokens_plus_meta(doc:spacy.tokens.doc.Doc):
    """Extract tokens and metadata from individual spaCy doc."""
    return [
        (i.text, i.i, i.lemma_, i.ent_type_, i.tag_, 
         i.dep_, i.pos_, i.is_stop, i.is_alpha, 
         i.is_digit, i.is_punct) for i in doc
    ]

def tidy_tokens(docs):
    """Extract tokens and metadata from list of spaCy docs."""
    
    cols = [
        "doc_id", "token", "token_order", "lemma", 
        "ent_type", "tag", "dep", "pos", "is_stop", 
        "is_alpha", "is_digit", "is_punct"
    ]
    
    meta_df = []
    for ix, doc in enumerate(docs):
        meta = extract_tokens_plus_meta(doc)
        meta = pd.DataFrame(meta)
        meta.columns = cols[1:]
        meta = meta.assign(doc_id = ix).loc[:, cols]
        meta_df.append(meta)
        
    return pd.concat(meta_df) 

I added a table to hold all the GOIDs and paths of the XMLs. I also rewrote the keyword search function so that each keyword could be searched in all the XMLs at once.

In [31]:
xml_info = pd.read_csv('xml_path.csv')
all_path = xml_info['Path'].tolist()
all_id = xml_info['GOID'].tolist()

In [36]:
def keyword_search(all_id, all_path, keyword):
    temp = pd.DataFrame()
    for i in range(len(all_path)):
        extracts = extract_paragraphs_new(all_path[i])
        extracts['GOID'] = all_id[i]
        extracts['Polarity'] = extracts['Text'].apply(get_polarity)
        extracts['Subjectivity'] = extracts['Text'].apply(get_subjectivity)
        temp = temp.append(extracts)
    result = temp[temp['Text'].str.contains(keyword)]
    return result        

In [37]:
keyword_search(all_id, all_path, 'Yellen')

,Paragraph,Sentence,Overall Sentence,Text,GOID,Polarity,Subjectivity
0,1.0,1.0,1.0,"Federal Reserve Chairwoman Janet Yellen pushed back against House Republicans' proposal to require the central bank to adopt a formal, mathematical rule to guide its interest-rate decisions.",1545387815,0.000000,0.083333
1,2.0,1.0,2.0,"Ms. Yellen, responding to some of the toughest questioning she has faced on Capitol Hill since becoming the Fed's leader in February, said provisions in a recently introduced House bill would lead to greater political meddling in the Fed's affairs, curtailing its independence and constraining the ability of policy makers to manage crises.",1545387815,0.237500,0.425000
2,3.0,1.0,3.0,"""It would be a grave mistake for the Fed to commit to conduct monetary policy according to a mathematical rule,"" Ms. Yellen told the House Financial Services Committee on Wednesday, amplifying her message in testimony to the Senate Banking Committee a day earlier.",1545387815,0.000000,0.166667
5,5.0,1.0,6.0,"Ms. Yellen defended the Fed's position of holding interest rates lower than such a rule would prescribe, saying, ""It is utterly necessary for us to provide more monetary-policy accommodation than those simple rules would have suggested.""",1545387815,0.125000,0.589286
0,1.0,1.0,1.0,"Janet L. Yellen, the Federal Reserve chairwoman, testified that a Republican proposal would undermine the central bank's independence in monetary policy.",1545553533,0.000000,0.250000
1,2.0,1.0,2.0,"The chairwoman of the Federal Reserve, Janet L. Yellen, says it would be a ""grave mistake"" for Congress to adopt legislation proposed by House Republicans to constrain the Fed's management of monetary policy.",1545553533,0.000000,0.000000
2,3.0,1.0,3.0,Ms. Yellen and Republican lawmakers traded strong words on Wednesday at a testy hearing before the House Financial Services Committee that focused on a bill that would require the Fed to publish a set of rules for raising and lowering interest rates.,1545553533,0.216667,0.366667
3,4.0,1.0,4.0,"Ms. Yellen said such a rule would impair the Fed's ability to manage the economy, even inhibiting its ability to respond to financial crises.",1545553533,0.000000,0.250000
5,5.0,1.0,6.0,Republicans criticized what they described as an alarmist response by Ms. Yellen and Democrats to a proposal that they said would do nothing more than modestly increase the Fed's accountability by requiring greater transparency.,1545553533,0.366667,0.633333
21,12.0,3.0,22.0,"Ms. Yellen did not give a direct answer on Wednesday to questions about how soon the Fed will begin to raise its key short-term gauge from its current rate near zero, but she did note that most economists expected the Fed to act in the third quarter of 2015.",1545553533,0.085714,0.442857


In [42]:
def get_number(all_id, all_path):
    temp = {'GOID': all_id, 'Paragraph Number': [], 'Sentence Number': []}
    for i in range(len(all_path)):
        extracts = extract_paragraphs_new(all_path[i])
        temp['Paragraph Number'].append(extracts['Paragraph'].max())
        temp['Sentence Number'].append(extracts['Overall Sentence'].max())
    df = pd.DataFrame(data=temp)   
    return df

In [43]:
get_number(all_id, all_path)

,GOID,Paragraph Number,Sentence Number
0,2123327034,29.0,97.0
1,1545387815,7.0,11.0
2,1545553533,18.0,33.0
3,1545554849,12.0,15.0
4,1546074706,5.0,5.0
5,1549576638,13.0,42.0
6,1549968105,29.0,46.0
7,1550716133,10.0,19.0
8,1550795074,12.0,31.0
9,1919515746,205.0,814.0


In [44]:
extract_paragraphs_new('RE_Sample_XMLs/304383476.xml')

,Paragraph,Sentence,Overall Sentence,Text


In [23]:
extracts_2123327034 = extract_paragraphs_new('RE_Sample_XMLs/2123327034.xml')
#extracts_2123327034['Tag'] = extracts_2123327034['Text'].apply(get_tags)
extracts_2123327034['Polarity'] = extracts_2123327034['Text'].apply(get_polarity)
extracts_2123327034['Subjectivity'] = extracts_2123327034['Text'].apply(get_subjectivity)
#extracts_2123327034.to_csv('extracts_2123327034.csv', index=False)

In [76]:
docs_2123327034 = list(nlp.pipe(extracts_2123327034['Text']))
nlp_info_2123327034 = tidy_tokens(docs_2123327034)
#nlp_info_2123327034.to_csv('nlp_info_2123327034.csv', index=False)

In [25]:
# Newspaper
extracts_1545387815 = extract_paragraphs_new('RE_Sample_XMLs/1545387815.xml')
#extracts_1545387815['Tag'] = extracts_1545387815['Text'].apply(get_tags)
extracts_1545387815['Polarity'] = extracts_1545387815['Text'].apply(get_polarity)
extracts_1545387815['Subjectivity'] = extracts_1545387815['Text'].apply(get_subjectivity)
#extracts_1545387815.to_csv('extracts_1545387815.csv', index=False)

In [26]:
yellen_1545387815 = extracts_1545387815[extracts_1545387815['Text'].str.contains('Yellen')]

In [79]:
extracts_1545387815[extracts_1545387815['Text'].str.contains('Wall Street')]

,Paragraph,Sentence,Overall Sentence,Text,Polarity,Subjectivity


In [80]:
docs_1545387815 = list(nlp.pipe(extracts_1545387815['Text']))
nlp_info_1545387815 = tidy_tokens(docs_1545387815)
nlp_info_1545387815.to_csv('Sample_Output/nlp_info_1545387815.csv', index=False)

In [81]:
# Newspaper
extracts_1545553533 = extract_paragraphs_new('RE_Sample_XMLs/1545553533.xml')
#extracts_1545553533['Tag'] = extracts_1545553533['Text'].apply(get_tags)
extracts_1545553533['Polarity'] = extracts_1545553533['Text'].apply(get_polarity)
extracts_1545553533['Subjectivity'] = extracts_1545553533['Text'].apply(get_subjectivity)

In [82]:
yellen_1545553533 = extracts_1545553533[extracts_1545553533['Text'].str.contains('Yellen')]

In [83]:
extracts_1545553533[extracts_1545553533['Text'].str.contains('Wall Street')]

,Paragraph,Sentence,Overall Sentence,Text,Polarity,Subjectivity


In [84]:
docs_1545553533 = list(nlp.pipe(extracts_1545553533['Text']))
nlp_info_1545553533 = tidy_tokens(docs_1545553533)
nlp_info_1545553533.to_csv('Sample_Output/nlp_info_1545553533.csv', index=False)

In [85]:
# Newspaper
extracts_1545554849 = extract_paragraphs_new('RE_Sample_XMLs/1545554849.xml')
#extracts_1545554849['Tag'] = extracts_1545554849['Text'].apply(get_tags)
extracts_1545554849['Polarity'] = extracts_1545554849['Text'].apply(get_polarity)
extracts_1545554849['Subjectivity'] = extracts_1545554849['Text'].apply(get_subjectivity)

In [86]:
yellen_1545554849 = extracts_1545554849[extracts_1545554849['Text'].str.contains('Yellen')]

In [87]:
extracts_1545554849[extracts_1545554849['Text'].str.contains('Wall Street')]

,Paragraph,Sentence,Overall Sentence,Text,Polarity,Subjectivity


In [88]:
docs_1545554849 = list(nlp.pipe(extracts_1545554849['Text']))
nlp_info_1545554849 = tidy_tokens(docs_1545554849)
nlp_info_1545554849.to_csv('Sample_Output/nlp_info_1545554849.csv', index=False)

In [90]:
# News
extracts_1550716133 = extract_paragraphs_new('RE_Sample_XMLs/1550716133.xml')
#extracts_1550716133['Tag'] = extracts_1550716133['Text'].apply(get_tags)
extracts_1550716133['Polarity'] = extracts_1550716133['Text'].apply(get_polarity)
extracts_1550716133['Subjectivity'] = extracts_1550716133['Text'].apply(get_subjectivity)

In [91]:
yellen_1550716133 = extracts_1550716133[extracts_1550716133['Text'].str.contains('Yellen')]

In [92]:
extracts_1550716133[extracts_1550716133['Text'].str.contains('Wall Street')]

,Paragraph,Sentence,Overall Sentence,Text,Polarity,Subjectivity


In [93]:
docs_1550716133 = list(nlp.pipe(extracts_1550716133['Text']))
nlp_info_1550716133 = tidy_tokens(docs_1550716133)
nlp_info_1550716133.to_csv('Sample_Output/nlp_info_1550716133.csv', index=False)

In [94]:
# Web
extracts_2046284852 = extract_paragraphs_new('RE_Sample_XMLs/2046284852.xml')
#extracts_2046284852['Tag'] = extracts_2046284852['Text'].apply(get_tags)
extracts_2046284852['Polarity'] = extracts_2046284852['Text'].apply(get_polarity)
extracts_2046284852['Subjectivity'] = extracts_2046284852['Text'].apply(get_subjectivity)

In [95]:
yellen_2046284852 = extracts_2046284852[extracts_2046284852['Text'].str.contains('Yellen')]

In [96]:
# keyword "Wall Street"
extracts_2046284852[extracts_2046284852['Text'].str.contains('Wall Street')]

,Paragraph,Sentence,Overall Sentence,Text,Polarity,Subjectivity
0,1,1,1,"Barbara Byrne, one of the most accomplished and powerful women in the banking industry, has seen a lot in her more than 30 years on Wall Street.",0.375000,0.625000
12,8,1,13,"Though promotion of women on Wall Street has grown by leaps and bounds over the past several decades, Byrne recalls that it wasn’t always that way.",-0.125000,0.125000
18,11,2,19,"In a recent interview, she discussed a range of issues including the changing gender landscape in the world of banking, the challenges and lessons of juggling motherhood and work, and advice she has for young women looking to make it on Wall Street.",0.050000,0.325000
19,12,1,20,Have Wall Street and the world of finance become more inclusive work environments for women over the past few decades?,0.016667,0.283333
25,15,1,26,Do you think it’s easier now to be a mother of four and work on Wall Street than it was 30 years ago?,0.000000,0.000000


In [97]:
docs_2046284852 = list(nlp.pipe(extracts_2046284852['Text']))
nlp_info_2046284852 = tidy_tokens(docs_2046284852)
nlp_info_2046284852.to_csv('Sample_Output/nlp_info_2046284852.csv', index=False)

In [98]:
# Concatenate all the result tables
yellen = pd.concat([yellen_1545387815, yellen_1545553533, yellen_1545554849, yellen_1550716133, yellen_2046284852], 
                   keys = [1545387815, 1545553533, 1545554849, 1550716133, 2046284852]).reset_index()

In [387]:
yellen = yellen.rename(columns={'level_0': 'GOID'})

In [388]:
yellen = yellen.drop(columns=['level_1'])

In [99]:
yellen.to_csv('Sample_Output/yellen.csv', index=False)